In [1]:
from database.scrape import scrape_defense, get_all_stats
import pandas as pd
import datetime
import re
import requests
from bs4 import BeautifulSoup as BS

In [2]:
def scrape_current_schedule(defense: str, date_year: int):
    """
    Scrapes defensive game-by-game statistics for a specific team in a given year from Pro Football Reference.

    This function sends an HTTP GET request to the specified URL for the team's defensive statistics for the given year,
    parses the HTML content, and returns a DataFrame with cleaned and standardized column names. The DataFrame
    includes statistics such as passing completions, attempts, yards, touchdowns, interceptions, and rushing attempts, yards, and touchdowns.

    Parameters:
    -----------
    year : int
        The year for which the defensive statistics are to be scraped (e.g., 2023).
    defense : str
        The abbreviation of the defensive team (e.g., "NE" for New England Patriots).

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the defensive team's game-by-game statistics for the specified year, with standardized column names.
    """
    URL = f"https://www.pro-football-reference.com/teams/{defense}/{date_year}/gamelog/"

    res = requests.get(URL, verify=False)

    soup = BS(res.content, "html.parser")

    table = soup.find_all("table", {"id": f"gamelog_opp{date_year}"})


    test = soup.find_all("td", {"data-stat": "opp"}, limit=17)

    code_list = []
    for i in test:
        html_string = str(i)

        # Regular expression to match the three-letter team code
        match = re.search(r"/teams/([a-z]{3})/", html_string)

        if match:
            team_code = match.group(1)
        else:
            print("No match found")
        code_list.append(team_code)

    df = pd.read_html(str(table))[0]
    flattened_columns = ["_".join(col).strip() for col in df.columns.values]
    df.columns = flattened_columns
    df.rename(
        columns={
            "Unnamed: 0_level_0_Week": "WEEK",
        },
        inplace=True,
    )
    df.columns = df.columns.str.upper()
    df.columns = df.columns.str.replace(".", "")
    df.columns = df.columns.str.replace("/", "_")
    df.columns = df.columns.str.replace(".", "")
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.replace("%", "_PCT")
    df["YEAR"] = date_year
    df["WEEK"] = df["WEEK"].astype(float)
    df["DEF_TEAM"] = defense.upper()
    df["OPP_CODE"] = code_list[:len(df["DEF_TEAM"])]
    df["OPP_CODE"] = df["OPP_CODE"].str.upper()

  
    df_def = df[
        [
            "DEF_TEAM",
            "OPP_CODE",
            "YEAR",
            "WEEK",
            "SCORE_OPP",
            "PASSING_CMP",
            "PASSING_ATT",
            "PASSING_YDS",
            "PASSING_TD",
            "PASSING_INT",
            "PASSING_SK",
            "PASSING_Y_A",
            "PASSING_NY_A",
            "PASSING_CMP_PCT",
            "PASSING_RATE",
            "RUSHING_ATT",
            "RUSHING_YDS",
            "RUSHING_Y_A",
            "RUSHING_TD",
        ]
    ]

    return df_def

In [3]:
scrape_current_schedule(defense='sfo', date_year=2020)

,DEF_TEAM,OPP_CODE,YEAR,WEEK,SCORE_OPP,PASSING_CMP,PASSING_ATT,PASSING_YDS,PASSING_TD,PASSING_INT,PASSING_SK,PASSING_Y_A,PASSING_NY_A,PASSING_CMP_PCT,PASSING_RATE,RUSHING_ATT,RUSHING_YDS,RUSHING_Y_A,RUSHING_TD
0,SFO,CRD,2020,1.0,24,26,40,224,1,1,2,5.8,5.3,65.0,77.5,36,180,5.0,2
1,SFO,NYJ,2020,2.0,13,21,32,173,1,0,1,5.6,5.2,65.6,89.7,29,104,3.6,0
2,SFO,NYG,2020,3.0,9,17,32,165,0,1,2,5.6,4.9,53.1,54.8,15,66,4.4,0
3,SFO,PHI,2020,4.0,25,18,28,174,1,1,3,6.9,5.6,64.3,78.6,28,93,3.3,1
4,SFO,MIA,2020,5.0,43,22,28,342,3,0,2,12.5,11.4,78.6,153.3,33,94,2.8,1
5,SFO,RAM,2020,6.0,16,19,38,198,2,1,0,5.2,5.2,50.0,72.0,19,113,5.9,0
6,SFO,NWE,2020,7.0,6,15,25,147,0,4,2,6.5,5.4,60.0,37.0,22,94,4.3,0
7,SFO,SEA,2020,8.0,37,27,37,249,4,0,2,7.1,6.4,73.0,127.0,28,101,3.6,1
8,SFO,GNB,2020,9.0,34,25,31,294,4,0,1,9.8,9.2,80.6,145.8,31,111,3.6,0
9,SFO,NOR,2020,10.0,27,14,23,123,1,0,3,6.0,4.7,60.9,89.6,30,114,3.8,2


In [4]:
pd.read_csv(r"database/DIM_DEFENSE.csv").drop_duplicates().to_csv(
    r"database/DIM_DEFENSE.csv"
)